In [1]:
#Import libraries
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.base import TaskResult

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [2]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential("your_github_token_here"),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "structured_output": True,
        "family": "unknown",
    },
)

In [3]:
frontdesk_agent = AssistantAgent(
    "planner_agent",
    model_client=client,
    description="A helpful assistant that can plan trips.",
    system_message="""
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.""",
)

concierge_agent = AssistantAgent(
    "concierge_agent",
    model_client=client,
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="""
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, respond with 'APPROVE'
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """,
)

In [7]:
termination = TextMentionTermination("APPROVE")
team = RoundRobinGroupChat(
    [frontdesk_agent, concierge_agent], termination_condition=termination
)

async for message in team.run_stream(task="I would like to plan a trip to Paris."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 6, 27, 10, 51, 40, 677270, tzinfo=datetime.timezone.utc) content='I would like to plan a trip to Paris.' type='TextMessage'
source='planner_agent' models_usage=RequestUsage(prompt_tokens=288, completion_tokens=17) metadata={} created_at=datetime.datetime(2025, 6, 27, 10, 51, 41, 705326, tzinfo=datetime.timezone.utc) content='Visit the Sainte-Chapelle for stunning stained glass windows and a quieter atmosphere.' type='TextMessage'
source='concierge_agent' models_usage=RequestUsage(prompt_tokens=327, completion_tokens=66) metadata={} created_at=datetime.datetime(2025, 6, 27, 10, 51, 44, 43164, tzinfo=datetime.timezone.utc) content="While Sainte-Chapelle is beautiful and less crowded than some major attractions, it can still be known to tourists. To enhance this recommendation, suggest exploring a hidden church or chapel with unique architectural features or local significance that may not be on the average tra